# 1. Set up enviroment and data

## 1.1. Import libraries

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns

## 1.2. Import data to dataframes

In [2]:
data1=pd.read_csv('./data/CusData.csv')
data1.drop(columns=["Unnamed: 0"], inplace=True)
data1.head()

,BET_ACCOUNT_NUM_HASH,AGE,AGE_BAND,GENDER,TENURE_IN_DAYS,RESIDENTIAL_STATE
0,13154,67.0,65+,M,11846,WA
1,18379,54.0,45-54,M,1884,WA
2,559232,63.0,55-64,M,2866,WA
3,698904,69.0,65+,M,2100,WA
4,762921,67.0,65+,M,4766,WA


In [ ]:
data2=pd.read_csv('./data/TransactionData.csv')
data2.head()

,Unnamed: 0,DATE_DIM,DAY_OF_WEEK,MONTH_OF_YEAR,BET_ACCOUNT_NUM_HASH,FOB_RACING_TURNOVER,FOB_SPORT_TURNOVER,PARI_RACING_TURNOVER,PARI_SPORT_TURNOVER,TOTAL_TURNOVER,DIVIDENDS_PAID,GROSS_MARGIN,TICKETS,YEAR
0,0,2021-01-01,Fri,1,13154,37.0,NaN,1081.0,NaN,1118.0,443.55,271.254275,288,2021
1,1,2021-01-01,Fri,1,18379,40.0,NaN,NaN,NaN,40.0,0.00,40.000000,1,2021
2,2,2021-01-01,Fri,1,559232,NaN,NaN,12.0,NaN,12.0,9.50,2.041720,5,2021
3,3,2021-01-01,Fri,1,698904,NaN,NaN,1223.5,NaN,1223.5,267.91,245.117147,40,2021
4,4,2021-01-01,Fri,1,762921,NaN,NaN,17.5,NaN,17.5,0.00,3.504075,5,2021


In [ ]:
data=pd.DataFrame(data2)
data3=pd.DataFrame(data2)

In [ ]:
data.drop(columns=["Unnamed: 0",'FOB_RACING_TURNOVER',	'FOB_SPORT_TURNOVER', 'PARI_RACING_TURNOVER', 'PARI_SPORT_TURNOVER', 'DIVIDENDS_PAID',	'GROSS_MARGIN',	'TICKETS'], inplace=True)
data.head()

,DATE_DIM,DAY_OF_WEEK,MONTH_OF_YEAR,BET_ACCOUNT_NUM_HASH,TOTAL_TURNOVER,YEAR
0,2021-01-01,Fri,1,13154,1118.0,2021
1,2021-01-01,Fri,1,18379,40.0,2021
2,2021-01-01,Fri,1,559232,12.0,2021
3,2021-01-01,Fri,1,698904,1223.5,2021
4,2021-01-01,Fri,1,762921,17.5,2021


# 2. RFM Calculation

In [ ]:
# Tìm ngày lớn nhất
data['DATE_DIM'] = pd.to_datetime(data['DATE_DIM'])
max_date = data['DATE_DIM'].max()
current_date = max_date + pd.Timedelta(days=1)
print("Ngày hiện tại:", current_date)

Ngày hiện tại: 2023-01-01 00:00:00


In [ ]:
# Tính toán Recency bằng cách tính số ngày kể từ ngày gần nhất
data['recency'] = (current_date - data['DATE_DIM']).dt.days
data.head()

,DATE_DIM,DAY_OF_WEEK,MONTH_OF_YEAR,BET_ACCOUNT_NUM_HASH,TOTAL_TURNOVER,YEAR,recency
0,2021-01-01,Fri,1,13154,1118.0,2021,730
1,2021-01-01,Fri,1,18379,40.0,2021,730
2,2021-01-01,Fri,1,559232,12.0,2021,730
3,2021-01-01,Fri,1,698904,1223.5,2021,730
4,2021-01-01,Fri,1,762921,17.5,2021,730


In [ ]:
# Tính toán Monetary
data['money'] = data['TOTAL_TURNOVER']
data.head()

,DATE_DIM,DAY_OF_WEEK,MONTH_OF_YEAR,BET_ACCOUNT_NUM_HASH,TOTAL_TURNOVER,YEAR,recency,money
0,2021-01-01,Fri,1,13154,1118.0,2021,730,1118.0
1,2021-01-01,Fri,1,18379,40.0,2021,730,40.0
2,2021-01-01,Fri,1,559232,12.0,2021,730,12.0
3,2021-01-01,Fri,1,698904,1223.5,2021,730,1223.5
4,2021-01-01,Fri,1,762921,17.5,2021,730,17.5


In [ ]:
# Tính toán Frequency bằng cách đếm số lần xuất hiện của mỗi BET_ACCOUNT_NUM_HASH
freq_data = data.groupby('BET_ACCOUNT_NUM_HASH').size().reset_index(name='freq')

In [ ]:
# Kết hợp dữ liệu recency, monetary và frequency lại với nhau
rfm_data = data.groupby('BET_ACCOUNT_NUM_HASH').agg({'recency': 'min', 'money': 'sum'}).reset_index()
rfm_data = pd.merge(rfm_data, freq_data, on='BET_ACCOUNT_NUM_HASH')
rfm_data[['recency', 'money', 'freq']] = rfm_data[['recency', 'money', 'freq']].astype(float)
# Hiển thị dữ liệu RFM
rfm_data.head()

,BET_ACCOUNT_NUM_HASH,recency,money,freq
0,13154,1.0,366186.08,463.0
1,18379,1.0,32033.00,199.0
2,37251,61.0,100.00,1.0
3,129273,13.0,155.00,16.0
4,348318,99.0,30.00,4.0


In [ ]:
rfm_data.describe()

,BET_ACCOUNT_NUM_HASH,recency,money,freq
count,1.448590e+05,144859.000000,1.448590e+05,144859.000000
mean,2.143528e+09,148.494936,1.515111e+04,85.332296
std,1.239167e+09,201.312452,6.188284e+04,142.057642
min,1.315400e+04,1.000000,0.000000e+00,1.000000
25%,1.071472e+09,5.000000,1.630000e+02,3.000000
50%,2.145177e+09,57.000000,9.920000e+02,18.000000
75%,3.211517e+09,246.000000,7.066750e+03,98.000000
max,4.294965e+09,730.000000,5.544861e+06,730.000000


# 3. RFM score

In [ ]:
rfm_c=rfm_data

In [ ]:
rfm_c['r_c'] = pd.qcut(rfm_c['recency'], 5, ['5','4','3','2','1'])
rfm_c['f_c'] = pd.qcut(rfm_c['freq'], 5, ['1','2','3','4','5'])
rfm_c['m_c'] = pd.qcut(rfm_c['money'], 5, ['1','2','3','4','5'])
rfm_c.head()

,BET_ACCOUNT_NUM_HASH,recency,money,freq,r_c,f_c,m_c
0,13154,1.0,366186.08,463.0,5,5,5
1,18379,1.0,32033.00,199.0,5,5,5
2,37251,61.0,100.00,1.0,3,1,1
3,129273,13.0,155.00,16.0,4,3,2
4,348318,99.0,30.00,4.0,2,2,1


In [ ]:
rfm_c['RFM_Score'] = rfm_c.r_c.astype(str)+ rfm_c.f_c.astype(str) + rfm_c.m_c.astype(str)
rfm_c.head()

,BET_ACCOUNT_NUM_HASH,recency,money,freq,r_c,f_c,m_c,RFM_Score
0,13154,1.0,366186.08,463.0,5,5,5,555
1,18379,1.0,32033.00,199.0,5,5,5,555
2,37251,61.0,100.00,1.0,3,1,1,311
3,129273,13.0,155.00,16.0,4,3,2,432
4,348318,99.0,30.00,4.0,2,2,1,221


In [ ]:
# Specify the file path where you want to save the CSV file
file_path = "./data/rfm_c.csv"
# Export merged data to CSV
rfm_c.to_csv(file_path, index=False)

In [ ]:
# Phân cụm theo recency, frequency và monetary và lưu trữ các cạnh bin
r_bins = pd.qcut(rfm_c['recency'], 5).unique().sort_values()
f_bins = pd.qcut(rfm_c['freq'], 5).unique().sort_values()
m_bins = pd.qcut(rfm_c['money'], 5).unique().sort_values()

# In ra các cạnh bin cho từng chỉ số RFM
print("Recency bin edges:", r_bins)
print("Frequency bin edges:", f_bins)
print("Monetary bin edges:", m_bins)

Recency bin edges: [(0.999, 2.0], (2.0, 25.0], (25.0, 61.0], (61.0, 359.0], (359.0, 730.0]]
Categories (5, interval[float64, right]): [(0.999, 2.0] < (2.0, 25.0] < (25.0, 61.0] < (61.0, 359.0] < (359.0, 730.0]]
Frequency bin edges: [(0.999, 2.0], (2.0, 10.0], (10.0, 35.0], (35.0, 137.0], (137.0, 730.0]]
Categories (5, interval[float64, right]): [(0.999, 2.0] < (2.0, 10.0] < (10.0, 35.0] < (35.0, 137.0] < (137.0, 730.0]]
Monetary bin edges: [(-0.001, 107.0], (107.0, 476.0], (476.0, 2128.2], (2128.2, 11181.438], (11181.438, 5544861.46]]
Categories (5, interval[float64, right]): [(-0.001, 107.0] < (107.0, 476.0] < (476.0, 2128.2] < (2128.2, 11181.438] < (11181.438, 5544861.46]]
